In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1), "T" => d[:total_tools], "N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.stan' at line 19)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE         MCSE         ESS    
         lp__ -39.442540350  3.1441776177 0.049713813200 0.13025247616  582.696175
accept_stat__   0.947379627  0.0748186849 0.001182987279 0.00156860829 1000.000000
   stepsize__   0.048285675  0.0040528888 0.000064081799 0.00064890047   39.009752
  treedepth__   5.938250000  0.4738069868 0.007491546248 0.02344356973  408.465553
 n_leapfrog__  75.932000000 36.7488366516 0.581050125903 1.71023237288  461.718148
  divergent__   0.000000000  0.0000000000 0.000000000000 0.00000000000         NaN
     energy__  45.930366950  4.0473606700 0.063994391147 0.15866713748  650.683570
        alpha   1.136873435  0.7099315461 0.011225003343 0.01770955966 1000.000000
  a_society.1  -0.204426819  0.2393983900 0.003785220904 0.00601795945 1000.000000
  a_society.2   0.036439649  0.2127763302 0.0033642891

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*